In [ ]:
# ---- Image Stitching 예제 9.5-1 프로그램

import numpy as np
import imutils
import cv2

ratio = 0.75 # ratio test 값
reprojThresh = 4.0 # inlier/outlier 판단 값

# 우측, 좌측 이미지 로드 후 리사이즈
imgL = cv2.imread('sedona_left_01.png') # train image
imgR = cv2.imread('sedona_right_01.png') # query image

imgL = imutils.resize(imgL, width=400)
imgR = imutils.resize(imgR, width=400)

descriptor = cv2.xfeatures2d.SIFT_create() # SIFT 객체 생성

# SIFT 방법으로 특징점 각각 추출
kpsL, featuresL = descriptor.detectAndCompute(imgL, None)
kpsR, featuresR = descriptor.detectAndCompute(imgR, None)

print('num of kpsL =', len(kpsL))
print('num of kpsR =', len(kpsR))

# 실수 출력 포맷 셋팅
float_formatter = "{:7.2f}".format
np.set_printoptions(formatter={'float_kind':float_formatter})

# Feature 좌표를 float 32bit 배열로 변환
kpsR = np.float32([kp.pt for kp in kpsR])
kpsL = np.float32([kp.pt for kp in kpsL])

matcher = cv2.BFMatcher() # Brute-force matcher 객체 생성

# descriptors 간에 전부 매칭 후, 
# 각각 가장 가까운 매칭점들을 2개씩 선별하여 리스트로 반환
matches = matcher.knnMatch(featuresR, featuresL, k=2)

print('\nnum of matches =', len(matches))

goodMatches = []      # 우수한 매칭점들을 저장할 리스트 생성

# ratio test로 우수한 매칭점인지 검사하여 goodMatches에 추가
for m in matches:
    # 가장 유사한 매칭점들이 2개 m[0]과 m[1] 비교
    if m[0].distance < m[1].distance * ratio :
        # 해당 매칭 좌표를 goodMatches에 입력
        goodMatches.append((m[0].queryIdx, m[0].trainIdx))

print('\nnum of goodMatches =', len(goodMatches))

# Homography 계산을 위해 X,Y 좌표 추출
ptsR = np.float32([kpsR[i] for (i, _) in goodMatches])
ptsL = np.float32([kpsL[i] for (_, i) in goodMatches])

# compute the homography between the two sets of points
(H, status) = cv2.findHomography(ptsR, ptsL, cv2.RANSAC, reprojThresh)

print('\nhomograpy matrix')
print(H)

print('\nlen(status) =', len(status))
print('sum of inliers =', sum(np.array(status).flatten()))

# 영상 스티칭을 위해 오른쪽 영상에 perspective warp 적용 
# 두 영상을 합치기 위해 width 추가
result = cv2.warpPerspective(imgR, H, (imgL.shape[1]+imgR.shape[1], 
                                       imgR.shape[0]))

result[0:imgL.shape[0], 0:imgL.shape[1]] = imgL # 왼쪽 이미지 삽입

# 좌측 이미지, 우측 이미지, Stitching 결과 이미지 출력
cv2.imshow("Left Image", imgL)
cv2.imshow("Right Image", imgR)
cv2.imshow("Stitched Image", result)

cv2.waitKey(0)
cv2.destroyAllWindows()
